<a href="https://colab.research.google.com/github/divgup/cs671/blob/master/assignment1_q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import h5py
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Activation,Dropout,Input
from keras.layers.normalization import BatchNormalization
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
with h5py.File('data.h5', 'r') as file:
    ids = pd.DataFrame(np.array(np.squeeze(file['ids'])),columns=['id'])
    full_data = np.array(file['images']).reshape(-1,28,28,3)
#label = pd.read_csv('label.csv')
w_label = label['wid']
l_label = label['len']
c_label = label['col']
a_label = label['ang']
a_label = pd.get_dummies(a_label)
#X_train,X_test,Y_train,Y1_train,Y_test,Y1_test = train_test_split(full_data,w_label,c_label,random_state=0,test_size=0.2)
X_train,X_test,w_train,w_test,c_train,c_test,l_train,l_test,a_train,a_test = train_test_split(full_data,w_label,c_label,l_label,a_label,random_state=42,test_size=0.2)
w_test = w_test.to_numpy()
c_test = c_test.to_numpy()
l_test = l_test.to_numpy()
a_test = a_test.to_numpy()

In [0]:
print(w_train.shape,c_train.shape,l_train.shape,a_train.shape)

In [0]:
#define seqeuntial model
inputs = Input(shape = (28,28,3,))
conv_1 = Conv2D(32,kernel_size=2,kernel_initializer='he_normal',input_shape=(28,28,3),activation='relu')(inputs)
batch_n1 = BatchNormalization()(conv_1)
drop_1 = Dropout(0.2)(batch_n1)
conv_2 = Conv2D(64,kernel_size=3,kernel_initializer='he_normal',activation='relu')(drop_1)
batch_n2 = BatchNormalization()(conv_2)
max_1 = MaxPooling2D((2,2))(batch_n2)
conv_3 = Conv2D(128,kernel_size=2,kernel_initializer='he_normal',activation='relu',padding='same')(conv_2)
batch_n3 = BatchNormalization()(conv_3)
max_2 = MaxPooling2D((2,2))(batch_n3)
conv_4 = Conv2D(64,kernel_size=2,kernel_initializer='he_normal',activation='relu')(conv_3)
batch_n4 = MaxPooling2D((2,2))(batch_n3)
max_3 = MaxPooling2D((2,2))(batch_n4)
drop_2 = Dropout(0.2)(max_3)
flat_1 = Flatten()(drop_2)

def angle_head():
  angle_d1 = Dense(512,activation='relu',kernel_initializer='glorot_uniform')
  angle_out = Dense(12,activation='softmax',name='angle_classifier')(angle_d1)
  return angle_out

def color_head():  
  color_d1 = Dense(512,activation='relu',kernel_initializer='glorot_uniform')(flat_1)
  color_out= Dense(1,activation='sigmoid',name='color_classifier')(color_d1)
  return color_out

def length_head():
  l_d1 = Dense(512,activation='relu')(flat_1)
  l_out = Dense(1,activation='sigmoid',name='length_classifier')(l_d1)
  return l_out

def width_head():
  w_d1 = Dense(512,activation='relu')(flat_1)
  w_out = Dense(1,activation='sigmoid',name='width_classifier')(w_d1)
  return w_out

width_h = width_head()
color_h = color_head()
length_h = length_head()
angle_h = angle_head()

model = Model(inputs,[w_h,color_h,length_h,angle_h])
model.summary()

In [0]:
no_epochs = 5
batch_size = 32
model.compile(optimizer='Adam',loss={'width_classifier':'binary_crossentropy','color_classifier':'binary_crossentropy','length_classifier':'binary_crossentropy','angle_classifier':'categorical_crossentropy'},loss_weights={'width_classifier':1.0,'color_classifier':1.0,'length_classifier':1.0,'angle_classifier':1.0},metrics=['accuracy'])
hist = model.fit(X_train,{'width_classifier':w_train,'color_classifier':c_train,'length_classifier':l_train,'angle_classifier':a_train},epochs = no_epochs,verbose=1)


In [0]:
total_loss,w_loss,c_closs,l_loss,a_loss,w_acc,c_acc,l_acc,a_acc = model.evaluate(X_test,[w_test,c_test,l_test,a_test],verbose=1)

In [0]:
def metrics(confmat):
    #confmat = confmat(true,pred)
    sum_diag = np.sum(np.diag(confmat))
    tot = np.sum(confmat)
    acc = sum_diag/tot
    precision = (np.diag(confmat))/np.sum(confmat,axis=1)
    recall = np.diag(confmat)/np.sum(confmat,axis=0)
    F_score = 2*precision*recall/(precision+recall)
    return acc,precision,recall,F_score

In [0]:
def confusionmatrix(true,pred):
    num_class = np.ndarray.max(true)+1
    confmat = [[0 for col in range(num_class)] for row in range(num_class)]
    for i in range(true.shape[0]):
        confmat[true[i]][pred[i]]+=1
    return confmat

In [0]:
pred = model.predict(X_test)
pred_w = pred[0].squeeze()
pred_c = pred[1].squeeze()
pred_l = pred[2].squeeze()
pred_a = pred[3].squeeze()
test_a = np.argmax(a_test,axis=1) 
pred_w[pred_w > 0.5] = 1
pred_w[pred_w < 0.5] = 0
pred_c[pred_c > 0.5] = 1
pred_c[pred_c < 0.5] = 0
pred_l[pred_l > 0.5] = 1
pred_l[pred_l < 0.5] = 0
pred_a = np.argmax(pred_a,axis=1)
pred_w = pred_w.astype('uint8')
pred_c = pred_c.astype('uint8')
pred_l = pred_l.astype('uint8')
pred_a = pred_a.astype('uint8')
#print(pred_w.shape)
print(test_a)
confmat_w = confusionmatrix(w_test,pred_w)
confmat_c = confusionmatrix(c_test,pred_c)
confmat_l = confusionmatrix(l_test,pred_l)
confmat_a = confusionmatrix(test_a,pred_a)


In [0]:
print(confmat_w)
print(confmat_a)
acc_w,prec_w,recall_w,f_score_w = metrics(confmat_w)
acc_c,prec_c,recall_c,f_score_w = metrics(confmat_c)
acc_l,prec_l,recall_l,f_score_l = metrics(confmat_l)
acc_a,prec_a,recall_a,f_score_a = metrics(confmat_a)

In [0]:
pred_label = pred_a*1+pred_l*12+pred_c*24+pred_w*48
true_label = test_a*1+l_test*12+c_test*24+w_test*48
confmat = confusionmatrix(true_label,pred_label)
#print(pred_label.shape)
#print(confmat)
acc,prec,recall,f_score = metrics(confmat)
print(acc,'\n')
print(prec)
#print(pred_label)

In [0]:
import matplotlib.pyplot as plt
def plot(epochs,hist):
    plt.style.use('ggplot')
    metrics=['width_classifier_loss','color_classifier_loss','length_classifier_loss','angle_classifier_loss']
    fig,ax = plt.subplots(2,2,figsize=(20,15))
    plt.subplots_adjust(hspace=0.2,wspace=0.2)
    for (i,l) in enumerate(metrics):
        title = 'Loss for {}'.format(l)
        ax[i//2,i%2].set_title(title,fontsize=15)
        ax[i//2,i%2].set_xlabel('Epoch',fontsize=12)
        ax[i//2,i%2].set_ylabel('loss',fontsize=12)
        ax[i//2,i%2].plot(np.arange(0,epochs),hist.history[l],label=l)
plot(no_epochs,hist)
print(hist.history)